In [1]:
# comma seperated values
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

def process_cpt_code(driver, cpt_code):
    wait = WebDriverWait(driver, 20)  # Increased timeout
    results = []  # Initialize the results list

    # Enter the CPT code into the search box
    time.sleep(5)
    try:
        search_box = wait.until(EC.presence_of_element_located((By.ID, 'tbxSearchBox')))
        search_box.clear()
        search_box.send_keys(cpt_code)
    except TimeoutException:
        print(f"Search box not found for CPT code: {cpt_code}")
        results.append({'cpt_code': cpt_code, 'article_id': '', 'keyword': '', 'similar_paragraphs': ''})  # Add CPT code with no article ID
        return results
    
    # Click the search button
    submit_button = driver.find_element(By.ID, 'btnSubmitSearch')
    submit_button.click()
    print(f"Submitted CPT code: {cpt_code}")
    
    time.sleep(10)

    # Wait for the search results to load
    try:
        wait.until(EC.visibility_of_element_located((By.ID, 'searchResultsDiv')))
        print("Search results loaded successfully.")
    except TimeoutException:
        print(f"Search results not loaded for CPT code: {cpt_code}")
        results.append({'cpt_code': cpt_code, 'article_id': '', 'keyword': '', 'similar_paragraphs': ''})  # Add CPT code with no article ID
        return results

    # Get the total number of results
    try:
        total_results_element = driver.find_element(By.ID, 'lblTotalResults')
        total_results = int(total_results_element.text)
        print(f"Total results for CPT code {cpt_code}: {total_results}")
    except ValueError:
        print(f"Invalid total results value for CPT code: {cpt_code}")
        results.append({'cpt_code': cpt_code, 'article_id': '', 'keyword': '', 'similar_paragraphs': ''})  # Add CPT code with no article ID
        return results

    # Flag to track if we have processed all articles
    processed_all_articles = False

    # Keywords to search within the article
    keywords = ['denied', 'non-covered', 'not covered', 'noncovered']

    # Initialize dictionaries to hold aggregated results
    article_ids = []
    keywords_found = []
    similar_paragraphs = []

    # Iterate through each result
    for i in range(total_results):
        if processed_all_articles:
            break  # If all articles are processed, exit the loop
        
        print(f"Processing article {i + 1} of {total_results} for CPT code: {cpt_code}")
        time.sleep(3)

        # Re-fetch article elements after each iteration
        article_elements = driver.find_elements(By.CLASS_NAME, 'table-title-col')
        if i < len(article_elements):
            try:
                # Scroll the article element into view to ensure it's clickable
                driver.execute_script("arguments[0].scrollIntoView();", article_elements[i])
                time.sleep(1)  # Allow some time for the page to settle
                
                # Try clicking the article using JavaScript to avoid interception
                driver.execute_script("arguments[0].click();", article_elements[i])
                time.sleep(1)  # Allow time for the article to load

                # Handle initial pop-ups (e.g., accept cookies)
                try:
                    accept_button = wait.until(EC.element_to_be_clickable((By.ID, 'btnAcceptLicense')))
                    accept_button.click()
                except TimeoutException:
                    pass

                # Wait for the article content to load and extract paragraphs
                wait.until(EC.presence_of_element_located((By.ID, 'h3ArticleGuidanceHeader')))
                
                # Use BeautifulSoup to parse the article page
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                paragraphs = soup.find_all('p')  # Extract all paragraphs
                
                # Track whether any keywords were found in the article
                article_keywords = []
                article_paragraphs = []
                for paragraph in paragraphs:
                    for keyword in keywords:
                        if re.search(r'\b{}\b'.format(re.escape(keyword)), paragraph.get_text(), re.IGNORECASE):
                            article_keywords.append(keyword)
                            article_paragraphs.append(paragraph.get_text())
                            break  # Stop after finding the first matching keyword in a paragraph

                if article_keywords:
                    article_id = driver.find_element(By.ID, 'lblTitleId').text
                    article_ids.append(article_id)
                    keywords_found.extend(article_keywords)
                    similar_paragraphs.extend(article_paragraphs)

            except Exception as e:
                print(f"Error processing article {i + 1}: {e}")

            finally:
                # After processing each article, check if all results are processed
                if i + 1 == total_results:
                    processed_all_articles = True  # We have processed all articles
                else:
                    # Return to search results page if not the last article
                    driver.back()
                    print(f"Returned to search results for CPT code: {cpt_code}")
                    time.sleep(0.1)

                    # Explicit wait to ensure the page reloads
                    wait.until(EC.visibility_of_element_located((By.ID, 'searchResultsDiv')))
                    time.sleep(1)
        else:
            print(f"No article element found for index {i}. Skipping.")

    # Aggregate results into single row for the CPT code
    if article_ids:
        results.append({
            'cpt_code': cpt_code, 
            'article_id': ', '.join(article_ids), 
            'keyword': ', '.join(keywords_found), 
            'similar_paragraphs': '\n\n'.join(similar_paragraphs)
        })
    else:
        results.append({'cpt_code': cpt_code, 'article_id': '', 'keyword': '', 'similar_paragraphs': ''})

    return results


def main():
    input_file = 'input.xlsx'
    output_file = 'output.xlsx'

    # Load the input Excel file
    df = pd.read_excel(input_file)

    # Ensure the CPT_CODE column is treated as a string
    df['CPT_CODE'] = df['CPT_CODE'].astype(str).apply(lambda x: x.zfill(5))

    # Setup Selenium WebDriver
    driver = webdriver.Chrome()  # Adjust the driver if needed (e.g., Edge, Firefox)

    # Process each CPT code
    all_results = []
    for _, row in df.iterrows():
        cpt_code = row['CPT_CODE']
        driver.get('abc.com')
        results = process_cpt_code(driver, cpt_code)
        if results:  # Check if results are not empty
            all_results.extend(results)

    # Save results to a new Excel file
    results_df = pd.DataFrame(all_results)
    results_df.to_excel(output_file, index=False)
    print("Results written to Excel successfully!")

    # Close the driver
    driver.quit()
    print("Process completed successfully!")


if __name__ == '__main__':
    main()


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# single line comments

import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

def process_cpt_code(driver, cpt_code):
    wait = WebDriverWait(driver, 20)  # Increased timeout
    results = []  # Initialize the results list

    # Enter the CPT code into the search box
    time.sleep(5)
    try:
        search_box = wait.until(EC.presence_of_element_located((By.ID, 'tbxSearchBox')))
        search_box.clear()
        search_box.send_keys(cpt_code)
    except TimeoutException:
        print(f"Search box not found for CPT code: {cpt_code}")
        results.append({'cpt_code': cpt_code, 'article_id': '', 'keyword': '', 'similar_paragraphs': ''})  # Add CPT code with no article ID
        return results
    
    # Click the search button
    submit_button = driver.find_element(By.ID, 'btnSubmitSearch')
    submit_button.click()
    print(f"Submitted CPT code: {cpt_code}")
    
    time.sleep(10)

    # Wait for the search results to load
    try:
        wait.until(EC.visibility_of_element_located((By.ID, 'searchResultsDiv')))
        print("Search results loaded successfully.")
    except TimeoutException:
        print(f"Search results not loaded for CPT code: {cpt_code}")
        results.append({'cpt_code': cpt_code, 'article_id': '', 'keyword': '', 'similar_paragraphs': ''})  # Add CPT code with no article ID
        return results

    # Get the total number of results
    try:
        total_results_element = driver.find_element(By.ID, 'lblTotalResults')
        total_results = int(total_results_element.text)
        print(f"Total results for CPT code {cpt_code}: {total_results}")
    except ValueError:
        print(f"Invalid total results value for CPT code: {cpt_code}")
        results.append({'cpt_code': cpt_code, 'article_id': '', 'keyword': '', 'similar_paragraphs': ''})  # Add CPT code with no article ID
        return results

    # Flag to track if we have processed all articles
    processed_all_articles = False

    # Keywords to search within the article
    keywords = ['denied', 'non-covered', 'not covered', 'noncovered']

    # Iterate through each result
    for i in range(total_results):
        if processed_all_articles:
            break  # If all articles are processed, exit the loop
        
        print(f"Processing article {i + 1} of {total_results} for CPT code: {cpt_code}")
        time.sleep(3)

        # Re-fetch article elements after each iteration
        article_elements = driver.find_elements(By.CLASS_NAME, 'table-title-col')
        if i < len(article_elements):
            try:
                # Scroll the article element into view to ensure it's clickable
                driver.execute_script("arguments[0].scrollIntoView();", article_elements[i])
                time.sleep(1)  # Allow some time for the page to settle
                
                # Try clicking the article using JavaScript to avoid interception
                driver.execute_script("arguments[0].click();", article_elements[i])
                time.sleep(1)  # Allow time for the article to load

                # Handle initial pop-ups (e.g., accept cookies)
                try:
                    accept_button = wait.until(EC.element_to_be_clickable((By.ID, 'btnAcceptLicense')))
                    accept_button.click()
                except TimeoutException:
                    pass

                # Wait for the article content to load and extract paragraphs
                wait.until(EC.presence_of_element_located((By.ID, 'h3ArticleGuidanceHeader')))
                
                # Use BeautifulSoup to parse the article page
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                paragraphs = soup.find_all('p')  # Extract all paragraphs
                
                similar_paragraphs = []
                for paragraph in paragraphs:
                    for keyword in keywords:
                        if re.search(r'\b{}\b'.format(re.escape(keyword)), paragraph.get_text(), re.IGNORECASE):
                            similar_paragraphs.append(paragraph.get_text())
                            # Store the keyword that matched
                            results.append({
                                'cpt_code': cpt_code, 
                                'article_id': driver.find_element(By.ID, 'lblTitleId').text, 
                                'keyword': keyword, 
                                'similar_paragraphs': paragraph.get_text()
                            })
                            break  # Stop after finding the first matching keyword in a paragraph

            except Exception as e:
                print(f"Error processing article {i + 1}: {e}")

            finally:
                # After processing each article, check if all results are processed
                if i + 1 == total_results:
                    processed_all_articles = True  # We have processed all articles
                else:
                    # Return to search results page if not the last article
                    driver.back()
                    print(f"Returned to search results for CPT code: {cpt_code}")
                    time.sleep(0.1)

                    # Explicit wait to ensure the page reloads
                    wait.until(EC.visibility_of_element_located((By.ID, 'searchResultsDiv')))
                    time.sleep(1)
        else:
            print(f"No article element found for index {i}. Skipping.")

    # If no articles are found for this CPT code, ensure it gets added with an empty article ID
    if not results:
        results.append({'cpt_code': cpt_code, 'article_id': '', 'keyword': '', 'similar_paragraphs': ''})

    return results


def main():
    input_file = 'input.xlsx'
    output_file = 'output.xlsx'

    # Load the input Excel file
    df = pd.read_excel(input_file)

    # Ensure the CPT_CODE column is treated as a string
    df['CPT_CODE'] = df['CPT_CODE'].astype(str)

    # Setup Selenium WebDriver
    driver = webdriver.Chrome()  # Adjust the driver if needed (e.g., Edge, Firefox)

    # Process each CPT code
    all_results = []
    for _, row in df.iterrows():
        cpt_code = row['CPT_CODE']
        driver.get('https://www.cms.gov/medicare-coverage-database/search.aspx')
        results = process_cpt_code(driver, cpt_code)
        if results:  # Check if results are not empty
            all_results.extend(results)

    # Save results to a new Excel file
    results_df = pd.DataFrame(all_results)
    results_df.to_excel(output_file, index=False)
    print("Results written to Excel successfully!")

    # Close the driver
    driver.quit()
    print("Process completed successfully!")


if __name__ == '__main__':
    main()